# Pipeline

This pipeline is intended to simplify to whole process of loading a dataset, creating gaps of different types in it, imputing the missing data and evaluating the imputation method.
You may want to edit the cells preceded by an <span style="color: red">**EDIT:**</span> sign to fit your needs.

## Install dependencies

In [ ]:
!pip install openpyxl > /dev/null 2>&1
!pip install jupyterlab-widgets > /dev/null 2>&1
!pip install jsfileupload > /dev/null 2>&1
!pip install pyxlsb > /dev/null 2>&1
!pip install sklearn > /dev/null 2>&1
!pip install scipy > /dev/null 2>&1

## Set an arbitrary random state

In [ ]:
import random
random_state = 0
random.seed(random_state)

## Load clean data

> The upload form only supports files up to 10Mo. For larger files, please upload them directly to JupyterHub and provide a relative link to them in the text input herebelow.

<span style="color:red">**EDIT**: customize the dataset configuration.</span>

In [ ]:
%run helpers/load_data.ipynb

config_select.value = "factory zero"
file_select.value = "054.xlsx"
# dfloader.add_features('flow_temp', sheet_name='flowHeatSpaceHeating')
dfloader.add_targets('power', sheet_name='smartMeter')


In [ ]:
df = dfloader.df
orig_size = len(df)
df = df.dropna()
size_after_drop = len(df)
print(f'Droping {orig_size - size_after_drop} rows (NaN values in features)')
df.head()

In [ ]:
df = df.head(10000)

## Create gaps of different sizes

<span style="color:red">**EDIT**: customize gaps sizes.</span>

In [ ]:
%run helpers/create_gaps.ipynb

## Run the imputation

<span style="color:red">**EDIT**: create your own imputation strategy.</span>

In [ ]:
%run helpers/impute.ipynb

## Evaluate the imputation results

In [ ]:
%run helpers/evaluate.ipynb

In [ ]:
%run helpers/saving.ipynb